In [ ]:
import json
import pandas
import time
import os
import recordlinkage
import jellyfish as jf
import csv

In [ ]:
outputDir = 'scraped_json//'
field_agg = "_"

## Load the source file dict
sourceFiles = json.loads(open('config//sourceFiles.json', "r").read())

## Load the file schema
sourceSchema = json.loads(open('config//sourceSchema.json', "r").read())

## Load the id config
idConfig = json.loads(open('config//idConfig.json', "r").read())

In [ ]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dict
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [ ]:
## Cycle through all of files per source and drop into a list of dicts
## Printing list names for reference sake
for key in sourceFiles.keys():
    print(key)
    vars()[key] = mergeSourceFiles(key, outputDir, sourceFiles)

sports247
rivals
cfbRef
allConf
allAmerican
nflData
ncaa


In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state") 

In [ ]:
#create a universal identifier
#CHANGE - this is not going to be a list
#CFB-Ref reports it in chronological order so we will join based on position 1

def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
createNewID(idConfig['sports247'], sports247, field_agg)
createNewID(idConfig['rivals'], rivals, field_agg)
createNewID(idConfig['cfbRef'], cfbRef, field_agg)
createNewID(idConfig['allConf'], allConf, field_agg)
createNewID(idConfig['allAmerican'], allAmerican, field_agg)
createNewID(idConfig['nflData'], nflData, field_agg)
createNewID(idConfig['ncaa'], ncaa, field_agg)

In [ ]:
df_sports247 = pandas.DataFrame(sports247)
df_rivals = pandas.DataFrame(rivals)
df_cfbRef = pandas.DataFrame(cfbRef)
df_allConf = pandas.DataFrame(allConf)
df_allAmerican = pandas.DataFrame(allAmerican)
df_nflData = pandas.DataFrame(nflData)
df_ncaa = pandas.DataFrame(ncaa)

In [ ]:
indexer = recordlinkage.Index()
indexer.block('school')
indexer.block('year')
candidate_links = indexer.index(df_sports247, df_rivals)

In [ ]:
c = recordlinkage.Compare()

c.exact('ID', 'ID', label='ID')

c.string('playerName', 'playerName', method='damerau_levenshtein', label='playerName')
c.string('city', 'city', label='city')
c.string('state', 'state', label='state')
c.string('highSchool', 'highSchool', label='highSchool')
c.string('position', 'position', label='position')

# The comparison vectors
features = c.compute(candidate_links, df_sports247, df_rivals)

features

ID  playerName      city  state  highSchool  position
0     19      0    0.222222  0.100000    0.0    0.208333       0.5
      34      0    0.000000  0.083333    0.0    0.083333       0.0
      43      0    0.083333  1.000000    1.0    1.000000       0.0
      51      0    0.250000  0.166667    0.0    0.090909       0.0
      55      0    0.250000  0.333333    0.0    0.105263       0.0
...          ..         ...       ...    ...         ...       ...
17063 16853   0    0.058824  0.222222    0.0    0.176471       0.0
      16855   0    0.235294  0.111111    0.0    0.200000       0.0
      16856   0    0.117647  0.333333    0.0    0.111111       0.0
      16861   0    0.352941  0.222222    0.0    0.111111       0.5
      16877   0    0.117647  0.111111    0.0    0.125000       0.5

[22388448 rows x 6 columns]

In [ ]:
features.describe()

,ID,playerName,city,state,highSchool
count,2.238845e+07,2.238845e+07,2.238845e+07,2.238845e+07,2.238845e+07
mean,7.146989e-04,1.540184e-01,1.343968e-01,1.668469e-01,1.253481e-01
std,2.672430e-02,9.673058e-02,1.214204e-01,3.415916e-01,9.999665e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,8.333333e-02,7.142857e-02,0.000000e+00,6.666667e-02
50%,0.000000e+00,1.538462e-01,1.111111e-01,0.000000e+00,1.176471e-01
75%,0.000000e+00,2.142857e-01,1.875000e-01,0.000000e+00,1.764706e-01
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
